In [22]:
import csv
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    """Check if the URL is valid."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def configure_browser():
    options = Options()
    options.add_argument("--headless")  # Enable headless mode
    options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
    options.add_argument("--window-size=1920x1080")  # Set window size
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--no-sandbox")  # Disable the sandbox for Chrome's renderer
    options.add_argument("--disable-images")  # Disable images
    options.page_load_strategy = 'eager'  # Use 'eager' page load strategy
    return webdriver.Chrome(options=options)

def find_google_oidc_on(website_url):
    logging.info(f"Processing website: {website_url}")
    driver = configure_browser()

    if not is_valid_url(website_url):
        logging.warning(f"Invalid URL: {website_url}")
        return website_url, False, "N/A"

    driver.get(website_url)
    logging.info("Website loaded")

    # Accept cookies
    cookie_buttons = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT", "AGREE", "Accept all cookies", "Accept All Cookies", "Allow", "ALLOW", "OK", "Ok", "I accept", "I Accept", "I ACCEPT"
    ]
    for button_text in cookie_buttons:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., '" + button_text + "')] | //a[contains(., '" + button_text + "')]"))
            )
            accept_button.click()
            break
        except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
            continue

    # Look for login section
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN", "Login/Register", "Account", "My Account"]
    login_found = False
    for login_pat in login_section:
        try:
            try:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
                )
            except TimeoutException:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{login_pat}') and (self::a or self::button)]"))
                )
            
            driver.execute_script("arguments[0].scrollIntoView(true);", login_button)
            try:
                login_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
            login_found = True
            logging.info(f"Login section found and clicked: {login_pat}")
            break
        except (TimeoutException, NoSuchElementException):
            logging.info(f"Login section not found: {login_pat}")
            continue

    if not login_found:
        logging.info("No login section found with standard patterns. Proceeding without login click.")

    # Google OIDC patterns and checks
    google_oidc_patterns = [
        "Sign in with Google", "Continue with Google","Sign In with Google", "Login with Google", 
        "Sign in using Google", "Log In with Google", "Login using Google",
        "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE"
    ]

    oidc_method = "N/A"
    redirect_or_element_found = False
    text_pattern_found = False

    try:
        # Wait for redirection or specific elements
        WebDriverWait(driver, 10).until(
          #  lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns) or driver.current_url != website_url
            lambda x: driver.current_url != website_url or any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        oidc_method = "Detected" if any(pattern in driver.page_source for pattern in google_oidc_patterns) else "N/A"
    except TimeoutException:
        logging.info("No OIDC redirect or element found.")
        redirect_or_element_found = True
    except TimeoutException:
        pass

    if redirect_or_element_found:
        if len(driver.window_handles) > 1:
            oidc_method = "Pop-up"
        elif driver.current_url != website_url:
            oidc_method = "Redirect URL"
        else:
            oidc_method = "Direct Link/Button"        
        
    has_google_oidc = oidc_method != "N/A"
   # has_google_oidc = check_for_google_oidc_patterns(driver, google_oidc_patterns)
    oidc_method = "Google OIDC" if has_google_oidc else "N/A"

    driver.quit()
    logging.info(f"Finished processing {website_url} : Google OIDC - {has_google_oidc}, Method - {oidc_method}")
    return website_url, has_google_oidc, oidc_method

    
def check_for_google_oidc_patterns(driver):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        return any(pattern in driver.page_source for pattern in google_oidc_patterns)
    except TimeoutException:
        return False

def process_websites(websites, max_threads=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(find_google_oidc_on, website): website for website in websites}

        for future in as_completed(futures):
            result = future.result()
            results.append(result)
    return results

def read_websites_from_csv(input_csv):
    websites = []
    with open(input_csv, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            websites.append(row[0])
    return websites

def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])

def main():
    input_csv = 'try8.csv'  
    output_csv = 'output-ds8.csv'

    websites_to_test = read_websites_from_csv(input_csv)
    results = process_websites(websites_to_test, max_threads=5)

    for website, uses_google_oidc, method in results:
        print(f"{website}: Google OIDC - {uses_google_oidc}, Method - {method}")
    write_results_to_csv(results, output_csv)

if __name__ == "__main__":
    main()

INFO:root:Processing website: ﻿https://www.abc.net.au
INFO:root:Processing website: https://www.canva.com/
INFO:root:Processing website: https://www.crazygames.com/
INFO:root:Processing website: https://www.duplichecker.com/
INFO:root:Processing website: https://www.imdb.com/
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Processing website: https://google.com
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section found and clicked: Log in
INFO:root:Login section not found: Log in
INFO:root:Finished processing https://www.crazygames.com/ : Google OIDC - True, Method - Google OIDC
INFO:root:Login section found and clicked: Sign in
INFO:root:Finished processing https://google.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://billboard.com
INFO:root:Website loaded
INFO:root:Processing website: https://kaigai-antenna.com
INFO

ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00007FF66C2C82B2+55298]
	(No symbol) [0x00007FF66C235E02]
	(No symbol) [0x00007FF66C0F045D]
	(No symbol) [0x00007FF66C132B4C]
	(No symbol) [0x00007FF66C1287BF]
	(No symbol) [0x00007FF66C1520AA]
	(No symbol) [0x00007FF66C1280CF]
	(No symbol) [0x00007FF66C1522C0]
	(No symbol) [0x00007FF66C16AAA4]
	(No symbol) [0x00007FF66C151E83]
	(No symbol) [0x00007FF66C12670A]
	(No symbol) [0x00007FF66C127964]
	GetHandleVerifier [0x00007FF66C640AAB+3694587]
	GetHandleVerifier [0x00007FF66C69728E+4048862]
	GetHandleVerifier [0x00007FF66C68F173+4015811]
	GetHandleVerifier [0x00007FF66C3647D6+695590]
	(No symbol) [0x00007FF66C240CE8]
	(No symbol) [0x00007FF66C23CF34]
	(No symbol) [0x00007FF66C23D062]
	(No symbol) [0x00007FF66C22D3A3]
	BaseThreadInitThunk [0x00007FFC578D257D+29]
	RtlUserThreadStart [0x00007FFC58A6AA58+40]


# ON Whole dataset

In [ ]:
import csv
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    """Check if the URL is valid."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def configure_browser():
    options = Options()
    options.add_argument("--headless")  # Enable headless mode
    options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
    options.add_argument("--window-size=1920x1080")  # Set window size
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--no-sandbox")  # Disable the sandbox for Chrome's renderer
    options.add_argument("--disable-images")  # Disable images
    options.page_load_strategy = 'eager'  # Use 'eager' page load strategy
    return webdriver.Chrome(options=options)

def find_google_oidc_on(website_url):
    logging.info(f"Processing website: {website_url}")
    driver = configure_browser()

    if not is_valid_url(website_url):
        logging.warning(f"Invalid URL: {website_url}")
        return website_url, False, "N/A"

    driver.get(website_url)
    logging.info("Website loaded")

    # Accept cookies
    cookie_buttons = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT", "AGREE", "Accept all cookies", "Accept All Cookies", "Allow", "ALLOW", "OK", "Ok", "I accept", "I Accept", "I ACCEPT"
    ]
    for button_text in cookie_buttons:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., '" + button_text + "')] | //a[contains(., '" + button_text + "')]"))
            )
            accept_button.click()
            break
        except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
            continue

    # Look for login section
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN", "Login/Register", "Account", "My Account"]
    login_found = False
    for login_pat in login_section:
        try:
            try:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
                )
            except TimeoutException:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{login_pat}') and (self::a or self::button)]"))
                )
            
            driver.execute_script("arguments[0].scrollIntoView(true);", login_button)
            try:
                login_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
            login_found = True
            logging.info(f"Login section found and clicked: {login_pat}")
            break
        except (TimeoutException, NoSuchElementException):
            logging.info(f"Login section not found: {login_pat}")
            continue

    if not login_found:
        logging.info("No login section found with standard patterns. Proceeding without login click.")

    # Google OIDC patterns and checks
    google_oidc_patterns = [
        "Sign in with Google", "Continue with Google","Sign In with Google", "Login with Google", 
        "Sign in using Google", "Log In with Google", "Login using Google","Sign up with Google",
        "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE"
    ]

    oidc_method = "N/A"
    redirect_or_element_found = False
    text_pattern_found = False

    try:
        # Wait for redirection or specific elements
        WebDriverWait(driver, 10).until(
          #  lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns) or driver.current_url != website_url
            lambda x: driver.current_url != website_url or any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        oidc_method = "Detected" if any(pattern in driver.page_source for pattern in google_oidc_patterns) else "N/A"
    except TimeoutException:
        logging.info("No OIDC redirect or element found.")
        redirect_or_element_found = True
    except TimeoutException:
        pass

    if redirect_or_element_found:
        if len(driver.window_handles) > 1:
            oidc_method = "Pop-up"
        elif driver.current_url != website_url:
            oidc_method = "Redirect URL"
        else:
            oidc_method = "Direct Link/Button"        
        
    has_google_oidc = oidc_method != "N/A"
   # has_google_oidc = check_for_google_oidc_patterns(driver, google_oidc_patterns)
    oidc_method = "Google OIDC" if has_google_oidc else "N/A"

    driver.quit()
    logging.info(f"Finished processing {website_url} : Google OIDC - {has_google_oidc}, Method - {oidc_method}")
    return website_url, has_google_oidc, oidc_method

def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])
    
def check_for_google_oidc_patterns(driver):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        return any(pattern in driver.page_source for pattern in google_oidc_patterns)
    except TimeoutException:
        return False

def process_websites(websites, max_threads=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(find_google_oidc_on, website): website for website in websites}

        for future in as_completed(futures):
            result = future.result()
            results.append(result)
    return results

def read_websites_from_csv(input_csv):
    websites = []
    with open(input_csv, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            websites.append(row[0])
    return websites

def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])

def main():
    input_csv = 'dataset.csv'  # Replace with your input CSV file name
    output_csv = 'dataset-output.csv'  # Replace with your output CSV file name

    websites_to_test = read_websites_from_csv(input_csv)
    results = process_websites(websites_to_test, max_threads=5)

    for website, uses_google_oidc, method in results:
        print(f"{website}: Google OIDC - {uses_google_oidc}, Method - {method}")
    write_results_to_csv(results, output_csv)

if __name__ == "__main__":
    main()


INFO:root:Processing website: ﻿Websites
INFO:root:Processing website: https://youtube.com
INFO:root:Processing website: https://google.com
INFO:root:Processing website: https://baidu.com
INFO:root:Processing website: https://bilibili.com
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Processing website: https://facebook.com
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Finished processing https://google.com : Google OIDC - False, Method - N/A
INFO:root:Finished processing https://youtube.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://qq.com
INFO:root:Processing website: https://twitter.com
INFO:root:Website loaded
INFO:root:Website loaded
IN

INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Finished processing https://bing.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Account
INFO:root:Processing website: https://yandex.ru
INFO:root:Website loaded
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Finished processing https://whatsapp.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://xvideos.com
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section found and clicked: Log in
INFO:root:Processing website: https://live.com
INFO:root:Finished processing https://openai.com : Google OIDC - False, Method - N/A
INFO:root:Website loaded
INFO:root:Processing website: https://pornhub.com
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section not found: Log in
INFO:

INFO:root:Finished processing https://csdn.net : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Sign in
INFO:root:Processing website: https://aliexpress.com
INFO:root:Website loaded
INFO:root:Login section not found: Sign In
INFO:root:Login section not found: Log In
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section not found: LOGON
INFO:root:Login section not found: Login
INFO:root:Login section not found: Log in
INFO:root:Login section not found: Log in
INFO:root:Login section not found: SIGN IN
INFO:root:Processing website: https://microsoftonline.com
INFO:root:Login section not found: Log in
INFO:root:Processing website: https://yahoo.co.jp
INFO:root:Website loaded
INFO:root:Login section not found: Sign in
INFO:root:Login section not found: LOGIN
INFO:root:Login section not found: Sign in
INFO:root:Login section not found: Sign In
INFO:root:Login section not found: LOG IN
INFO:root:Login section not found

import csv
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    """Check if the URL is valid."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def configure_browser():
    options = Options()
    options.add_argument("--headless")  # Enable headless mode
    options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
    options.add_argument("--window-size=1920x1080")  # Set window size
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--no-sandbox")  # Disable the sandbox for Chrome's renderer
    options.add_argument("--disable-images")  # Disable images
    options.page_load_strategy = 'eager'  # Use 'eager' page load strategy
    return webdriver.Chrome(options=options)

def find_google_oidc_on(website_url):
    logging.info(f"Processing website: {website_url}")
    driver = configure_browser()

    if not is_valid_url(website_url):
        logging.warning(f"Invalid URL: {website_url}")
        return website_url, False, "N/A"

    driver.get(website_url)
    logging.info("Website loaded")

    # Accept cookies
    cookie_buttons = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT", "AGREE", "Accept all cookies", "Accept All Cookies", "Allow", "ALLOW", "OK", "Ok", "I accept", "I Accept", "I ACCEPT"
    ]
    for button_text in cookie_buttons:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., '" + button_text + "')] | //a[contains(., '" + button_text + "')]"))
            )
            accept_button.click()
            break
        except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
            continue

    # Look for login section
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN", "Login/Register", "Account", "My Account"]
    login_found = False
    for login_pat in login_section:
        try:
            try:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
                )
            except TimeoutException:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{login_pat}') and (self::a or self::button)]"))
                )
            
            driver.execute_script("arguments[0].scrollIntoView(true);", login_button)
            try:
                login_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
            login_found = True
            logging.info(f"Login section found and clicked: {login_pat}")
            break
        except (TimeoutException, NoSuchElementException):
            logging.info(f"Login section not found: {login_pat}")
            continue

    if not login_found:
        logging.info("No login section found with standard patterns. Proceeding without login click.")

    # Google OIDC patterns and checks
    google_oidc_patterns = [
        "Sign in with Google", "Continue with Google","Sign In with Google", "Login with Google", 
        "Sign in using Google", "Log In with Google", "Login using Google","Sign up with Google",
        "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE"
    ]

    oidc_method = "N/A"
    redirect_or_element_found = False
    text_pattern_found = False

    try:
        # Wait for redirection or specific elements
        WebDriverWait(driver, 10).until(
          #  lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns) or driver.current_url != website_url
            lambda x: driver.current_url != website_url or any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )

        redirect_or_element_found = True
    except TimeoutException:
        pass
    
        if any(pattern in driver.page_source for pattern in google_oidc_patterns):
            text_pattern_found = True    
    
    if redirect_or_element_found:
        if len(driver.window_handles) > 1:
            oidc_method = "Pop-up"
        elif driver.current_url != website_url:
            oidc_method = "Redirect URL"
        else:
            oidc_method = "Direct Link/Button"        
        
    has_google_oidc = oidc_method != "N/A"
   # has_google_oidc = check_for_google_oidc_patterns(driver, google_oidc_patterns)
  #  oidc_method = "Google OIDC" if has_google_oidc else "N/A"

    driver.quit()
    return has_google_oidc, oidc_method
    logging.info(f"Finished processing {website_url} : Google OIDC - {has_google_oidc}, Method - {oidc_method}")
    print(f"{website_url} : Google OIDC - {has_google_oidc}, Method - {oidc_method}")
    return website_url, has_google_oidc, oidc_method

def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])
    
def check_for_google_oidc_patterns(driver):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        return any(pattern in driver.page_source for pattern in google_oidc_patterns)
    except TimeoutException:
        return False

def process_websites(websites, max_threads=10):
    results = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(find_google_oidc_on, website): website for website in websites}

        for future in as_completed(futures):
            result = future.result()
            results.append(result)
    return results

def read_websites_from_csv(input_csv):
    websites = []
    with open(input_csv, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            websites.append(row[0])
    return websites

def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])

def main():
    input_csv = 'try8.csv'  # Replace with your input CSV file name
    output_csv = 'dataset-output.csv'  # Replace with your output CSV file name

    websites_to_test = read_websites_from_csv(input_csv)
    results = process_websites(websites_to_test, max_threads=10)

    for website, uses_google_oidc, method in results:
        print(f"{website}: Google OIDC - {uses_google_oidc}, Method - {method}")
    write_results_to_csv(results, output_csv)

if __name__ == "__main__":
    main()


In [7]:
import csv
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    """Check if the URL is valid."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def configure_browser():
    options = Options()
    options.add_argument("--headless")  # Enable headless mode
    options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
    options.add_argument("--window-size=1920x1080")  # Set window size
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--no-sandbox")  # Disable the sandbox for Chrome's renderer
    options.add_argument("--disable-images")  # Disable images
    options.page_load_strategy = 'eager'  # Use 'eager' page load strategy
    return webdriver.Chrome(options=options)

def find_google_oidc_on(website_url):
    logging.info(f"Processing website: {website_url}")
    driver = configure_browser()

    if not is_valid_url(website_url):
        logging.warning(f"Invalid URL: {website_url}")
        return website_url, False, "N/A"

    driver.get(website_url)
    logging.info("Website loaded")

    # Accept cookies
    cookie_buttons = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT", "AGREE", "Accept all cookies", "Accept All Cookies", "Allow", "ALLOW", "OK", "Ok", "I accept", "I Accept", "I ACCEPT"
    ]
    for button_text in cookie_buttons:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., '" + button_text + "')] | //a[contains(., '" + button_text + "')]"))
            )
            accept_button.click()
            break
        except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
            continue

    # Look for login section
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN", "Login/Register", "Account", "My Account"]
    login_found = False
    for login_pat in login_section:
        try:
            try:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
                )
            except TimeoutException:
                login_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{login_pat}') and (self::a or self::button)]"))
                )
            
            driver.execute_script("arguments[0].scrollIntoView(true);", login_button)
            try:
                login_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
            login_found = True
            logging.info(f"Login section found and clicked: {login_pat}")
            break
        except (TimeoutException, NoSuchElementException):
            logging.info(f"Login section not found: {login_pat}")
            continue

    if not login_found:
        logging.info("No login section found with standard patterns. Proceeding without login click.")

    # Google OIDC patterns and checks
    google_oidc_patterns = [
        "Sign in with Google", "Continue with Google","Sign In with Google", "Login with Google", 
        "Sign in using Google", "Log In with Google", "Login using Google","Sign up with Google",
        "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE"
    ]

    oidc_method = "N/A"
    redirect_or_element_found = False
    text_pattern_found = False

    try:
        # Wait for redirection or specific elements
        WebDriverWait(driver, 10).until(
          #  lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns) or driver.current_url != website_url
            lambda x: driver.current_url != website_url or any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        oidc_method = "Detected" if any(pattern in driver.page_source for pattern in google_oidc_patterns) else "N/A"
    except TimeoutException:
        logging.info("No OIDC redirect or element found.")
        redirect_or_element_found = True
    except TimeoutException:
        pass

    if redirect_or_element_found:
        if len(driver.window_handles) > 1:
            oidc_method = "Pop-up"
        elif driver.current_url != website_url:
            oidc_method = "Redirect URL"
        else:
            oidc_method = "Direct Link/Button"        
        
    has_google_oidc = oidc_method != "N/A"
   # has_google_oidc = check_for_google_oidc_patterns(driver, google_oidc_patterns)
    oidc_method = "Google OIDC" or "Pop-Up" or "Redirect URL" if has_google_oidc else "N/A"

    driver.quit()
    logging.info(f"Finished processing {website_url} : Google OIDC - {has_google_oidc}, Method - {oidc_method}")
    return website_url, has_google_oidc, oidc_method

def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])
    
def check_for_google_oidc_patterns(driver):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        return any(pattern in driver.page_source for pattern in google_oidc_patterns)
    except TimeoutException:
        return False

def process_websites(websites, max_threads=8):
    results = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(find_google_oidc_on, website): website for website in websites}

        for future in as_completed(futures):
            result = future.result()
            results.append(result)
    return results

def read_websites_from_csv(input_csv):
    websites = []
    with open(input_csv, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            websites.append(row[0])
    return websites

def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])

def main():
    input_csv = 'try8.csv'  # Replace with your input CSV file name
    output_csv = 'what8.csv'  # Replace with your output CSV file name

    websites_to_test = read_websites_from_csv(input_csv)
    results = process_websites(websites_to_test, max_threads=8)

    for website, uses_google_oidc, method in results:
        print(f"{website}: Google OIDC - {uses_google_oidc}, Method - {method}")
    write_results_to_csv(results, output_csv)

if __name__ == "__main__":
    main()


INFO:root:Processing website: ﻿https://www.canva.com/
INFO:root:Processing website: https://www.crazygames.com/
INFO:root:Processing website: https://www.duplichecker.com/
INFO:root:Processing website: https://www.imdb.com/
INFO:root:Processing website: https://google.com
INFO:root:Processing website: https://tiktok.com
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Finished processing https://www.crazygames.com/ : Google OIDC - True, Method - Google OIDC
INFO:root:Finished processing https://google.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login sec

﻿https://www.canva.com/: Google OIDC - False, Method - N/A
https://www.crazygames.com/: Google OIDC - True, Method - Google OIDC
https://google.com: Google OIDC - False, Method - N/A
https://tiktok.com: Google OIDC - True, Method - Google OIDC
https://www.duplichecker.com/: Google OIDC - True, Method - Google OIDC
https://www.imdb.com/: Google OIDC - True, Method - Google OIDC


In [8]:
import csv
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    """Check if the URL is valid."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def configure_browser():
    options = Options()
    options.add_argument("--headless")  # Enable headless mode
    options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
    options.add_argument("--window-size=1920x1080")  # Set window size
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--no-sandbox")  # Disable the sandbox for Chrome's renderer
    options.add_argument("--disable-images")  # Disable images
    options.page_load_strategy = 'eager'  # Use 'eager' page load strategy
    return webdriver.Chrome(options=options)

def find_google_oidc_on(website_url):
    logging.info(f"Processing website: {website_url}")
    driver = configure_browser()

    if not is_valid_url(website_url):
        logging.warning(f"Invalid URL: {website_url}")
        return website_url, False, "N/A"

    driver.get(website_url)
    logging.info("Website loaded")

    # Accept cookies
    cookie_buttons = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT", "AGREE", "Accept all cookies", "Accept All Cookies", "Allow", "ALLOW", "OK", "Ok", "I accept", "I Accept", "I ACCEPT"
    ]
    for button_text in cookie_buttons:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., '" + button_text + "')] | //a[contains(., '" + button_text + "')]"))
            )
            accept_button.click()
            break
        except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
            continue

    # Look for login section
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN", "Login/Register", "Account", "My Account"]
    login_found = False
    for login_pat in login_section:
        try:
            try:
                login_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
                )
            except TimeoutException:
                login_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{login_pat}') and (self::a or self::button)]"))
                )
            
            driver.execute_script("arguments[0].scrollIntoView(true);", login_button)
            try:
                login_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
            login_found = True
            logging.info(f"Login section found and clicked: {login_pat}")
            break
        except (TimeoutException, NoSuchElementException):
            logging.info(f"Login section not found: {login_pat}")
            continue

    if not login_found:
        logging.info("No login section found with standard patterns. Proceeding without login click.")

    # Google OIDC patterns and checks
    google_oidc_patterns = [
        "Sign in with Google", "Continue with Google","Sign In with Google", "Login with Google", 
        "Sign in using Google", "Log In with Google", "Login using Google","Sign up with Google",
        "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE"
    ]

    oidc_method = "N/A"
    redirect_or_element_found = False
    text_pattern_found = False

    try:
        # Wait for redirection or specific elements
        WebDriverWait(driver, 10).until(
          #  lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns) or driver.current_url != website_url
            lambda x: driver.current_url != website_url or any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        oidc_method = "Detected" if any(pattern in driver.page_source for pattern in google_oidc_patterns) else "N/A"
    except TimeoutException:
        logging.info("No OIDC redirect or element found.")
        redirect_or_element_found = True
    except TimeoutException:
        pass

    if redirect_or_element_found:
        if len(driver.window_handles) > 1:
            oidc_method = "Pop-up"
        elif driver.current_url != website_url:
            oidc_method = "Redirect URL"
        else:
            oidc_method = "Direct Link/Button"        
        
    has_google_oidc = oidc_method != "N/A"
   # has_google_oidc = check_for_google_oidc_patterns(driver, google_oidc_patterns)
    oidc_method = "Google OIDC" or "Pop-Up" or "Redirect URL" if has_google_oidc else "N/A"

    driver.quit()
    logging.info(f"Finished processing {website_url} : Google OIDC - {has_google_oidc}, Method - {oidc_method}")
    return website_url, has_google_oidc, oidc_method

##def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])
"""   
def check_for_google_oidc_patterns(driver):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        return any(pattern in driver.page_source for pattern in google_oidc_patterns)
    except TimeoutException:
        return False
""" 
def process_websites(websites, max_threads=8):
    results = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(find_google_oidc_on, website): website for website in websites}

        for future in as_completed(futures):
            result = future.result()
            results.append(result)
    return results
 


def read_websites_from_csv(input_csv):
    websites = []
    with open(input_csv, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            websites.append(row[0])
    return websites


def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])

def main():
    input_csv = 'try8.csv'  # Replace with your input CSV file name
    output_csv = '8.csv'  # Replace with your output CSV file name

    websites_to_test = read_websites_from_csv(input_csv)
    results = process_websites(websites_to_test, max_threads=8)

    for website, uses_google_oidc, method in results:
        print(f"{website}: Google OIDC - {uses_google_oidc}, Method - {method}")
    write_results_to_csv(results, output_csv)

if __name__ == "__main__":
    main()


INFO:root:Processing website: ﻿https://www.canva.com/
INFO:root:Processing website: https://www.crazygames.com/
INFO:root:Processing website: https://www.duplichecker.com/
INFO:root:Processing website: https://www.imdb.com/
INFO:root:Processing website: https://google.com
INFO:root:Processing website: https://tiktok.com
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section found and clicked: Log in
INFO:root:Finished processing https://www.crazygames.com/ : Google OIDC - True, Method - Google OIDC
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Finished processing https://google.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login sec

﻿https://www.canva.com/: Google OIDC - False, Method - N/A
https://www.crazygames.com/: Google OIDC - True, Method - Google OIDC
https://google.com: Google OIDC - False, Method - N/A
https://tiktok.com: Google OIDC - True, Method - Google OIDC
https://www.duplichecker.com/: Google OIDC - True, Method - Google OIDC
https://www.imdb.com/: Google OIDC - True, Method - Google OIDC


# Trying on Complete DB

In [ ]:
import csv
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

logging.basicConfig(level=logging.INFO)

def is_valid_url(url):
    """Check if the URL is valid."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

def configure_browser():
    options = Options()
    options.add_argument("--headless")  # Enable headless mode
    options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
    options.add_argument("--window-size=1920x1080")  # Set window size
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--no-sandbox")  # Disable the sandbox for Chrome's renderer
    options.add_argument("--disable-images")  # Disable images
    options.page_load_strategy = 'eager'  # Use 'eager' page load strategy
    return webdriver.Chrome(options=options)

def find_google_oidc_on(website_url):
    logging.info(f"Processing website: {website_url}")
    driver = configure_browser()

    if not is_valid_url(website_url):
        logging.warning(f"Invalid URL: {website_url}")
        return website_url, False, "N/A"

    driver.get(website_url)
    logging.info("Website loaded")

    # Accept cookies
    cookie_buttons = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT", "AGREE", "Accept all cookies", "Accept All Cookies", "Allow", "ALLOW", "OK", "Ok", "I accept", "I Accept", "I ACCEPT"
    ]
    for button_text in cookie_buttons:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., '" + button_text + "')] | //a[contains(., '" + button_text + "')]"))
            )
            accept_button.click()
            break
        except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
            continue

    # Look for login section
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN", "Login/Register", "Account", "My Account"]
    login_found = False
    for login_pat in login_section:
        try:
            try:
                login_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
                )
            except TimeoutException:
                login_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//*[contains(text(), '{login_pat}') and (self::a or self::button)]"))
                )
            
            driver.execute_script("arguments[0].scrollIntoView(true);", login_button)
            try:
                login_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
            login_found = True
            logging.info(f"Login section found and clicked: {login_pat}")
            break
        except (TimeoutException, NoSuchElementException):
            logging.info(f"Login section not found: {login_pat}")
            continue

    if not login_found:
        logging.info("No login section found with standard patterns. Proceeding without login click.")

    # Google OIDC patterns and checks
    google_oidc_patterns = [
        "Sign in with Google", "Continue with Google","Sign In with Google", "Login with Google", 
        "Sign in using Google", "Log In with Google", "Login using Google","Sign up with Google",
        "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE","Sign In With Google"
    ]

    oidc_method = "N/A"
    redirect_or_element_found = False
    text_pattern_found = False

    try:
        # Wait for redirection or specific elements
        WebDriverWait(driver, 10).until(
          #  lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns) or driver.current_url != website_url
            lambda x: driver.current_url != website_url or any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        oidc_method = "Detected" if any(pattern in driver.page_source for pattern in google_oidc_patterns) else "N/A"
    except TimeoutException:
        logging.info("No OIDC redirect or element found.")
        redirect_or_element_found = True
    except TimeoutException:
        pass

    if redirect_or_element_found:
        if len(driver.window_handles) > 1:
            oidc_method = "Pop-up"
        elif driver.current_url != website_url:
            oidc_method = "Redirect URL"
        else:
            oidc_method = "Direct Link/Button"        
        
    has_google_oidc = oidc_method != "N/A"
   # has_google_oidc = check_for_google_oidc_patterns(driver, google_oidc_patterns)
    oidc_method = "Google OIDC" or "Pop-Up" or "Redirect URL" if has_google_oidc else "N/A"

    driver.quit()
    logging.info(f"Finished processing {website_url} : Google OIDC - {has_google_oidc}, Method - {oidc_method}")
    return website_url, has_google_oidc, oidc_method

##def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])
"""   
def check_for_google_oidc_patterns(driver):
    try:
        WebDriverWait(driver, 10).until(
            lambda x: any(pattern in driver.page_source for pattern in google_oidc_patterns)
        )
        return any(pattern in driver.page_source for pattern in google_oidc_patterns)
    except TimeoutException:
        return False
""" 
def process_websites(websites, max_threads=8):
    results = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(find_google_oidc_on, website): website for website in websites}

        for future in as_completed(futures):
            result = future.result()
            results.append(result)
    return results
 


def read_websites_from_csv(input_csv):
    websites = []
    with open(input_csv, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            websites.append(row[0])
    return websites


def write_results_to_csv(results, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Website', 'Google OIDC Present', 'Method'])
        for website, uses_google_oidc, method in results:
            csv_writer.writerow([website, uses_google_oidc, method])

def main():
    input_csv = 'dataset.csv'  # Replace with your input CSV file name
    output_csv = 'dataset-result.csv'  # Replace with your output CSV file name

    websites_to_test = read_websites_from_csv(input_csv)
    results = process_websites(websites_to_test, max_threads=8)

    for website, uses_google_oidc, method in results:
        print(f"{website}: Google OIDC - {uses_google_oidc}, Method - {method}")
    write_results_to_csv(results, output_csv)

if __name__ == "__main__":
    main()


INFO:root:Processing website: ﻿Websites
INFO:root:Processing website: https://google.com
INFO:root:Processing website: https://youtube.com
INFO:root:Processing website: https://baidu.com
INFO:root:Processing website: https://bilibili.com
INFO:root:Processing website: https://facebook.com
INFO:root:Processing website: https://qq.com
INFO:root:Processing website: https://twitter.com
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Processing website: https://zhihu.com
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Website loaded
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Finished processing https://google.com : Google OIDC - False, Method - N/A
INFO:root:Finis

INFO:root:Login section not found: Login
INFO:root:Processing website: https://pornhub.com
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Processing website: https://microsoft.com
INFO:root:Login section not found: Log in
INFO:root:Website loaded
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:Finished processing https://live.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Sign in
INFO:root:Processing website: https://vk.com
INFO:root:Website loaded
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section found and clicked: Log in
INFO:root:Login section not found: Sign In
INFO:root:Finished processing https://openai.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://zoom.us
INFO:root:Website loaded
INFO:root:Login section not found: Log in
INFO:root:Login section found and clicked: Sign in
INFO:root:F

INFO:root:Login section not found: LOGIN
INFO:root:Website loaded
INFO:root:Login section not found: Sign in
INFO:root:Login section not found: Log in
INFO:root:Login section not found: LOG IN
INFO:root:Login section not found: Sign in
INFO:root:Processing website: https://xhamster.com
INFO:root:Login section not found: SIGN IN
INFO:root:Website loaded
INFO:root:Login section not found: LOG IN
INFO:root:Login section not found: Sign In
INFO:root:Login section not found: Login/Register
INFO:root:Login section not found: Sign In
INFO:root:Login section not found: LOGIN
INFO:root:Login section not found: Login/Register
INFO:root:Login section not found: Log In
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login
INFO:root:Login section not found: Account
INFO:root:Login section not found: Log In
INFO:root:Login section not found: LOG IN
INFO:root:Login section not found: Account
INFO:root:Login section not found: LOGON
INFO:root:Login section not found: Log in

INFO:root:Finished processing https://spankbang.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://telegram.org
INFO:root:Login section not found: LOGIN
INFO:root:Website loaded
INFO:root:Login section not found: Sign in
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Finished processing https://iqiyi.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Sign in
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Processing website: https://dzen.ru
INFO:root:Finished processing https://mail.ru : Google OIDC - False, Method - N/A
INFO:root:Website loaded
INFO:root:Processing website: https://quora.com
INFO:root:Login section not found: LOG IN
INFO:root:Website loaded
INFO:root:Login section not found: Sign In
INFO:root:Login section not found: Login
INF

INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Finished processing https://spotify.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: LOGON
INFO:root:Processing website: https://feishu.cn
INFO:root:Website loaded
INFO:root:Login section not found: Log In
INFO:root:Login section not found: Account
INFO:root:Login section not found: SIGN IN
INFO:root:Login section not found: Login
INFO:root:Login section not found: LOGON
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Finished processing https://sohu.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Login
INFO:root:Processing website: https://duckduckgo.com
INFO:root:Website loaded
INFO:root:Login section not found: LOGIN
INFO:root:Login section not found: Log in
INFO:root:Login section not found: SIGN IN
INFO:root:Login section not found: Log

...

INFO:root:Login section not found: LOG IN
INFO:root:Login section not found: Account
INFO:root:Login section not found: SIGN IN
INFO:root:Processing website: https://homedepot.com
INFO:root:Login section not found: LOG IN
INFO:root:Website loaded
INFO:root:Login section not found: Account
INFO:root:Login section not found: Login
INFO:root:Login section not found: Login/Register
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Login section not found: LOGIN
INFO:root:Login section not found: Login/Register
INFO:root:Finished processing https://gofile.io : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://sahibinden.com
INFO:root:Login section not found: Account
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Login section not found: Log in
INFO:root:Login section not found: L

INFO:root:Finished processing https://sahibinden.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://dribbble.com
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
ERROR:selenium.webdriver.common.service:Service process refused to terminate gracefully with SIGTERM, escalating to SIGKILL.
Traceback (most recent call last):
  File "C:\Users\Nainsi\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 168, in _terminate_process
    self.process.wait(60)
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 1591, in _wait
    raise TimeoutExpired(self.args, timeout)
subprocess.TimeoutExpired: Command '['C:\\Users\\Nainsi\\.cache\\selenium\\chromedriver\\win64\\119.0.6045.105\\chromedriver.exe'

INFO:root:Login section not found: Log in
INFO:root:Login section not found: Log in
INFO:root:Login section not found: Account
INFO:root:Login section not found: SIGN IN
INFO:root:Login section not found: Sign in
INFO:root:Finished processing https://marca.com : Google OIDC - False, Method - N/A
INFO:root:Login section not found: Sign in
INFO:root:Login section not found: Sign in
INFO:root:Processing website: https://gamer.com.tw
INFO:root:Login section not found: LOGIN
INFO:root:Login section not found: Sign In
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Login section not found: Sign In
INFO:root:Login section not found: Sign In
INFO:root:Website loaded
INFO:root:Login section not found: LOG IN
INFO:root:Login section not found: Log In
INFO:root:Login section not found: Log In
INFO:root:Login section not found: Log In
INFO:root:Login section not found: Login
INFO:root:Login section not

INFO:root:Finished processing https://dongchedi.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://softonic.com
ERROR:selenium.webdriver.common.service:Service process refused to terminate gracefully with SIGTERM, escalating to SIGKILL.
Traceback (most recent call last):
  File "C:\Users\Nainsi\AppData\Roaming\Python\Python311\site-packages\selenium\webdriver\common\service.py", line 168, in _terminate_process
    self.process.wait(60)
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\subprocess.py", line 1591, in _wait
    raise TimeoutExpired(self.args, timeout)
subprocess.TimeoutExpired: Command '['C:\\Users\\Nainsi\\.cache\\selenium\\chromedriver\\win64\\119.0.6045.105\\chromedriver.exe', '--port=10120']' timed out after 60 seconds
INFO:root:Finished processing https://google.cn : Google OIDC - False, Method - N/A
INFO:r

INFO:root:Finished processing https://trendyol.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://9animetv.to
INFO:root:Login section not found: Log In
INFO:root:Login section not found: LOGON
INFO:root:Login section not found: SIGN IN
INFO:root:Processing website: https://tokopedia.com
INFO:root:Login section not found: LOGIN
INFO:root:Login section not found: LOG IN
INFO:root:Login section not found: Login/Register
INFO:root:Login section not found: Account
INFO:root:Login section not found: My Account
INFO:root:No login section found with standard patterns. Proceeding without login click.
INFO:root:Login section not found: LOGIN
INFO:root:Finished processing https://lanhuapp.com : Google OIDC - False, Method - N/A
INFO:root:Processing website: https://excretekings.com
INFO:root:Login section not found: LOG IN
INFO:root:Login section not found: Login/Register
INFO:root:Login section not found: Account
INFO:root:Login section not found: My Account
INFO:root: